# Importing some libraries

In [ ]:
import os
import wget
import numpy as np
import pandas as pd
import soundfile as sf
from zipfile import ZipFile
import matplotlib.pyplot as plt
%matplotlib inline

# Downloading Dataset

In [ ]:
dataset_url = 'https://bhichallenge.med.auth.gr/sites/default/files/ICBHI_final_database/ICBHI_final_database.zip'
wget.download(dataset_url)

In [ ]:
with ZipFile('ICBHI_final_database.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall()
os.remove('ICBHI_final_database.zip')

In [ ]:
root = 'ICBHI_final_database/'
filenames = [s.split('.')[0] for s in os.listdir(path = root) if '.txt' in s]
# Just patient info
filenames = filenames[:920]

In [ ]:
def Extract_Annotation_Data(file_name, root):
    tokens = file_name.split('_')
    recording_info = pd.DataFrame(data = [tokens], columns = ['Patient number', 'Recording index', 'Chest location','Acquisition mode','Recording equipment'])
    recording_annotations = pd.read_csv(os.path.join(root, file_name + '.txt'), names = ['Start', 'End', 'Crackles', 'Wheezes'], delimiter= '\t')
    return (recording_info, recording_annotations)

In [ ]:
i_list = []
rec_annotations = []
rec_annotations_dict = {}
for s in filenames:
    (i,a) = Extract_Annotation_Data(s, root)
    i_list.append(i)
    rec_annotations.append(a)
    rec_annotations_dict[s] = a
recording_info = pd.concat(i_list, axis = 0)
recording_info.head()

In [ ]:
no_label_list = []
crack_list = []
wheeze_list = []
both_sym_list = []
filename_list = []
for f in filenames:
    d = rec_annotations_dict[f]
    no_labels = len(d[(d['Crackles'] == 0) & (d['Wheezes'] == 0)].index)
    n_crackles = len(d[(d['Crackles'] == 1) & (d['Wheezes'] == 0)].index)
    n_wheezes = len(d[(d['Crackles'] == 0) & (d['Wheezes'] == 1)].index)
    both_sym = len(d[(d['Crackles'] == 1) & (d['Wheezes'] == 1)].index)
    no_label_list.append(no_labels)
    crack_list.append(n_crackles)
    wheeze_list.append(n_wheezes)
    both_sym_list.append(both_sym)
    filename_list.append(f)

In [ ]:
file_label_df = pd.DataFrame(data = {'filename':filename_list, 'no label':no_label_list, 'crackles only':crack_list, 'wheezes only':wheeze_list, 'crackles and wheezees':both_sym_list})
file_label_df.head()

# Distribution of data classes

In [ ]:
file_label_df.sum()

In [ ]:
file_label_df.iloc[919, file_label_df.columns.get_loc('filename')] = "226_1b1_Pl_sc_Meditron"

In [ ]:
file_label_df.insert(1, 'id', recording_info['Patient number'].tolist())
file_label_df.head()

## Downloading Train/Test split

In [ ]:
split_url = 'https://bhichallenge.med.auth.gr/sites/default/files/ICBHI_final_database/ICBHI_challenge_train_test.txt'
train_test_df = pd.read_table(split_url, delimiter='\t', names=['filename', 'set'])
train_test_df.head()

In [ ]:
file_label_df = file_label_df.merge(train_test_df, 'left')
file_label_df.head()

## Spliting

In [ ]:
train_only = file_label_df[file_label_df['set']=='train']
train_only.head()

In [ ]:
train_only.shape

### Validation

In [ ]:
# Patients ID list, you can choose different ones
val_id = ['112','132','138','163','166','221']

In [ ]:
val_df = train_only.loc[(train_only.id.isin(val_id))]
val_df.head()

### Train

In [ ]:
train_df = train_only.drop(val_df.index.values, axis=0)
train_df.head()

### Test

In [ ]:
test_df = file_label_df[file_label_df['set']=='test']

In [ ]:
test_df.shape

# Distribution of respiratory cycle lengths

In [ ]:
duration_list = []
for i in range(len(rec_annotations)):
    current = rec_annotations[i]
    duration = current['End'] - current['Start']
    duration_list.extend(duration)

duration_list = np.array(duration_list)
plt.hist(duration_list, bins = 50)
print('longest cycle:{:.2f}'.format(max(duration_list)))
print('shortest cycle:{:.2f}'.format(min(duration_list)))
print('mean cycle length:{:.2f}'.format(np.mean(duration_list)))
threshold = np.mean(duration_list)
print('Number of samples less than {:.2f} seconds:{:.2f}'.format(threshold,
                                                           np.sum(duration_list < threshold)))

print('Number of samples higher than {:.2f} seconds:{:.2f}'.format(threshold,
                                                           np.sum(duration_list > threshold)))

# Data preparation utility functions

In [ ]:
#Used to split each individual sound file into separate sound clips containing one respiratory cycle each
#output: [filename, (sample_data:np.array, start:float, end:float, crackles:bool(float), wheezes:bool(float)) (...) ]
import math
import librosa

def read_wav_file(str_filename, target_rate):
    data, sample_rate = librosa.load(str_filename, sr=None)
        
    if (sample_rate != target_rate):
        data = librosa.resample(y=data, orig_sr=sample_rate, target_sr=target_rate)
    
    return (target_rate, data.astype(np.float32))

def slice_data(start, end, raw_data,  sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    return raw_data[start_ind: end_ind]

def get_sound_samples(recording_annotations, file_name, root, sample_rate):
    sample_data = [file_name]
    (rate, data) = read_wav_file(os.path.join(root, file_name + '.wav'), sample_rate)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row['Start']
        end = row['End']
        crackles = row['Crackles']
        wheezes = row['Wheezes']
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start, end, crackles, wheezes))
    return sample_data

#Fits each respiratory cycle into a fixed length audio clip, splits may be performed and zero padding is added if necessary
#original:(arr,c,w) -> output:[(arr,c,w),(arr,c,w)]
def split_and_pad(original, desiredLength, sampleRate):
    output_buffer_length = int(desiredLength * sampleRate)
    soundclip = original[0]
    n_samples = len(soundclip)
    total_length = n_samples / sampleRate #length of cycle in seconds
    n_slices = int(math.ceil(total_length / desiredLength)) #get the minimum number of slices needed
    samples_per_slice = n_samples // n_slices
    src_start = 0 #Staring index of the samples to copy from the original buffer
    output = [] #Holds the resultant slices
    for i in range(n_slices):
        src_end = min(src_start + samples_per_slice, n_samples)
        length = src_end - src_start
        copy = generate_padded_samples(soundclip[src_start:src_end], output_buffer_length)
        output.append((copy, original[1], original[2]))
        src_start += length
    return output

def generate_padded_samples(source, output_length):
    copy = np.zeros(output_length, dtype = np.float32)
    src_length = len(source)
    if(output_length > src_length):
        #tile forward sounds to fill empty space
        cursor = 0
        while(cursor + src_length) < output_length:
            copy[cursor:(cursor + src_length)] = source[:]
            cursor += src_length
    else:
        copy[:src_length] = source[:]
    #
    return copy

In [ ]:
#Takes a list of respiratory cycles, and splits and pads each cycle into fixed length buffers (determined by desiredLength(seconds))
#Then takes the split and padded sample and transforms it into a mel spectrogram

def apply_split_and_pad(original, desiredLength, sampleRate):
    output = []
    for d in original:
        
        lst_result = split_and_pad(d, desiredLength, sampleRate) #Time domain
        output.extend(lst_result)
        
    return output

# Utility used to import all samples

In [ ]:
def extract_all_samples(filenames, annotation_dict, root, target_rate, desired_length):
    cycle_list = []
    for file in filenames:
        data = get_sound_samples(annotation_dict[file], file, root, target_rate)
        cycles_with_labels = [(d[0], d[3], d[4]) for d in data[1:]]
        cycle_list.extend(cycles_with_labels)
    
    #Sort into respective classes
    no_labels = [c for c in cycle_list if ((c[1] == 0) & (c[2] == 0))]
    c_only = [c for c in cycle_list if ((c[1] == 1) & (c[2] == 0))] 
    w_only = [c for c in cycle_list if ((c[1] == 0) & (c[2] == 1))]
    c_w = [c for c in cycle_list if ((c[1] == 1) & (c[2] == 1))]
    
    #Split up cycles into sound clips with fixed lengths so they can be fed into a CNN
    
    none  = (apply_split_and_pad(no_labels, desired_length, target_rate))
    
    c = (apply_split_and_pad(c_only, desired_length, target_rate))
    
    w = (apply_split_and_pad(w_only, desired_length, target_rate))
    
    c_w = (apply_split_and_pad(c_w, desired_length, target_rate)) 
    
    dict = {'none':none,'crackles':c,'wheezes':w, 'both':c_w}

    return dict

# Function return number of samples 
def print_sample_count(src_dict):
    print('none:{}\ncrackles:{}\nwheezes:{}\nboth:{}'.format(len(src_dict['none']),
                                                        len(src_dict['crackles']),
                                                        len(src_dict['wheezes']),
                                                        len(src_dict['both'])))
    

# Extracting Samples per set 

In [ ]:
# Parameters
target_sample_rate = 32000
sample_length_seconds = 10

In [ ]:
# train 
sample_dict_train = extract_all_samples(train_df.filename.tolist(), rec_annotations_dict, root, target_sample_rate, sample_length_seconds) 

In [ ]:
# test
sample_dict_test = extract_all_samples(test_df.filename.tolist(), rec_annotations_dict, root, target_sample_rate, sample_length_seconds) 

In [ ]:
# validation
sample_dict_val = extract_all_samples(val_df.filename.tolist(), rec_annotations_dict, root, target_sample_rate, sample_length_seconds) 

In [ ]:
print('Samples Available')
print('[Training set]')
print_sample_count(sample_dict_train)

In [ ]:
print('Samples Available')
print('[Testing set]')
print_sample_count(sample_dict_test)

In [ ]:
print('Samples Available')
print('[Validation set]')
print_sample_count(sample_dict_val)

## Saving wav files

In [ ]:
# Saving Train samples
dir = 'ICBHI/Train'
os.makedirs(dir, exist_ok=True)
train_list = []

for sample in ['none', 'crackles', 'wheezes', 'both']:
    # Saving Normal Samples
    for i, file in enumerate(sample_dict_train[sample]):
        name = sample+"_"+str(i)+".wav"
        x = file[0]
        path = os.path.join(dir, name)
        sf.write(path, x, target_sample_rate)

        if sample=='none':
            train_dict = {'filename':name, 'id':0}
        elif sample=='crackles':
            train_dict = {'filename':name, 'id':1}
        elif sample=='wheezes':
            train_dict = {'filename':name, 'id':2}
        else:
            train_dict = {'filename':name, 'id':4}
            
        train_list.append(train_dict)

In [ ]:
# Creating dataframe for train samples and save It
train_samples = pd.DataFrame(train_list, columns = train_dict.keys())
train_samples.to_csv(dir+'.csv', index=False)

In [ ]:
# Saving Test samples
test_dir = 'ICBHI/Test'
os.makedirs(test_dir, exist_ok=True)
test_list = []

for sample in ['none', 'crackles', 'wheezes', 'both']:
    for i, file in enumerate(sample_dict_test[sample]):
        name = sample+"_"+str(i)+".wav"
        x = file[0]
        path = os.path.join(test_dir, name)
        sf.write(path, x, target_sample_rate)
        
        if sample=='none':
            test_dict = {'filename':name, 'id':0}
        elif sample=='crackles':
            test_dict = {'filename':name, 'id':1}
        elif sample=='wheezes':
            test_dict = {'filename':name, 'id':2}
        else:
            test_dict = {'filename':name, 'id':4}
        
        test_list.append(test_dict)

In [ ]:
# Creating dataframe for Test samples and save It 
test_samples = pd.DataFrame(test_list, columns = test_dict.keys())
test_samples.to_csv(test_dir+'.csv', index=False)

In [ ]:
# Saving Validation samples
val_dir = 'ICBHI/Val'
os.makedirs(val_dir, exist_ok=True)
val_list = []

# Saving Normal Samples
for sample in ['none', 'crackles', 'wheezes', 'both']:
    for i, file in enumerate(sample_dict_val[sample]):
        name = sample+"_"+str(i)+".wav"
        x = file[0]
        path = os.path.join(val_dir, name)
        sf.write(path, x, target_sample_rate)
        
        if sample=='none':
            val_dict = {'filename':name, 'id':0}
        elif sample=='crackles':
            val_dict = {'filename':name, 'id':1}
        elif sample=='wheezes':
            val_dict = {'filename':name, 'id':2}
        else:
            val_dict = {'filename':name, 'id':4}
        
        val_list.append(val_dict)

In [ ]:
# Creating dataframe for validation samples nd save It 
val_samples = pd.DataFrame(val_list, columns = val_dict.keys())
val_samples.to_csv(val_dir+'.csv', index=False)

In [ ]:
os.remove('ICBHI_final_database')